In [10]:
import numpy as np
import itertools as it
from PGM_PyLib.MRF import RMRFwO as mrf
import requests
import pandas as pd
import contextlib
import io

In [11]:
# Function to generate random probability distributions for bounding boxes
def generate_random_probabilities(num_boxes, num_states):
    """
    Generate random probability distributions for each bounding box.
    
    Parameters:
    - num_boxes: int, number of bounding boxes
    - num_states: int, number of states (e.g., "tree", "glass", "car", "cup")

    Returns:
    - numpy array of shape (num_boxes, num_states), where each row is a
      probability distribution over the states for a bounding box.
    """
    probabilities = np.random.rand(num_boxes, num_states)  # Generate random values
    probabilities /= probabilities.sum(axis=1, keepdims=True)  # Normalize each row to sum to 1
    return probabilities

# Generate the 4x4 relational probability matrix (for the 4 states)
def generate_symmetric_matrix(n):
    """Generate an n x n symmetric matrix for 0-based indexing."""
    mat = np.random.rand(n, n)  # Generate random values
    mat = (mat + mat.T) / 2  # Make the matrix symmetric
    np.fill_diagonal(mat, 1)  # Fill the diagonal with 1s for self-relations
    return mat

# Define psi and Uf functions
def psi(state_i, state_j):
    """
    Return the probability of transitioning from state_i to state_j
    based on the relational probability matrix (0-based indexing).
    """
    return relational_probabilities[state_i, state_j]

# Assuming relational_probabilities is global
def Uf(rmrf, observation, row, col):
    """
    Local energy function for a bounding box in a complete graph (3 rows, 1 column RMRF).
    
    This function calculates the energy based on the current state of the bounding box
    and its relationships with all other bounding boxes (complete graph assumption).
    Additionally, it considers n-tuples (n >= 3) by averaging the weights for all such tuples.
    Observation probabilities are also factored into the energy calculation.
    
    Parameters:
    - rmrf: 2D numpy array, current RMRF matrix (3 rows, 1 column, states should be 0-based)
    - observation: bounding box probability distributions
    - row: int, row index (0-based, corresponding to the bounding box)
    - col: int, column index (should always be 0 since it's a 1-column matrix)

    Returns:
    - float, the calculated energy for the bounding box at row `row`
    """
    current_state = rmrf[row, col]  # Get the current state of the bounding box
    energy = 0.0
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_prob = observation[row][current_state]  # Probability for the current state
    energy -= np.log(obs_prob)  # Add the energy contribution from the observation
    
    # 2. Calculate the pairwise (2-tuple) energy from the relationships with other bounding boxes (complete graph)
    num_boxes = rmrf.shape[0]  # Number of bounding boxes (rows in rmrf)

    for other_row in range(num_boxes):
        if other_row != row:  # Avoid self-relations
            other_state = rmrf[other_row, col]  # Get the state of the other bounding box
            # Get the potential (weight) from the relational probability matrix
            weight = relational_probabilities[current_state, other_state]
            energy -= np.log(weight)  # Add the energy contribution from the relationship

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples (where n >= 3)
    for n in range(3, num_boxes + 1):  # Start from n=3, up to n=num_boxes
        # Get all combinations of bounding boxes of size n
        tuples = list(it.combinations(range(num_boxes), n))
        
        # For each n-tuple, calculate the average relational probability
        for t in tuples:
            # Extract the relational probabilities for all pairs in the tuple
            pairwise_weights = [
                relational_probabilities[rmrf[i, col], rmrf[j, col]]
                for i, j in it.combinations(t, 2)
            ]
            
            # Calculate the mean of the pairwise weights using NumPy
            if pairwise_weights:  # Ensure there are weights to average
                average_weight = np.mean(pairwise_weights)
                energy -= np.log(average_weight)  # Subtract the energy from the average weight
    
    return energy

In [34]:
def global_energy(current_states, observation, relational_matrix):
    """
    Global energy function for the entire RMRF in the current state (complete graph assumption).
    """

    energy           = 0.0
    num_vertices     = len(current_states) # Number of bounding boxes
    pairwise_weights = relational_matrix[np.ix_(current_states, current_states)] # Compute the pairwise weights by indexing the relational_matrix
    
    # 1. Factor in the observation (1-tuple) likelihood for the bounding box
    obs_probs    = observation[np.arange(num_vertices), current_states]  # Extract the probabilities for the current states
    energy      -= np.sum(np.log(obs_probs)) # Sum the log probabilities and subtract from energy
    
    # 2. Consider 2-tuples and calculate the weight as basis for the energy
    i_upper, j_upper = np.triu_indices(num_vertices, k=1)  # Get indices for all 2-tuples (upper triangle)
    energy          -= np.sum(np.log(pairwise_weights[i_upper, j_upper]))  # Subtract log of all pairwise weights

    # 3. Consider n-tuples (n >= 3) and calculate the average weight as basis for the energy
    # Iterate over all possible n-tuples, starting from n=3
    for n in range(3, num_vertices + 1):
        # Get all n-tuples (combinations of vertices of size n)
        tuples = list(it.combinations(range(num_vertices), n))
        
        # Collect pairwise weights for all tuples of this size n
        all_pairwise_weights = []
        
        for t in tuples:
            # Extract the pairwise weights for the current n-tuple (submatrix)
            submatrix = pairwise_weights[np.ix_(t, t)]  # Submatrix for the current n-tuple
            i, j = np.triu_indices(n, k=1)  # Indices for upper triangular part (pairwise combinations)
            tuple_weights = submatrix[i, j]  # Pairwise weights for this n-tuple
            
            # Add the weights to the list of all pairwise weights for this n
            all_pairwise_weights.extend(tuple_weights)
        
        # Calculate the mean over all pairwise weights across all n-tuples
        if all_pairwise_weights:  # Ensure there are weights to average
            average_weight = np.mean(all_pairwise_weights)
            energy -= np.log(average_weight)  # Subtract log of the mean of all tuple weights
    
    return energy

In [33]:
current_states = np.array([0, 1, 2])  # Random initial current_states
observation = bounding_box_probabilities
energy = 0.0
num_vertices = len(current_states)
relational_matrix = relational_probabilities
print(relational_probabilities)


pairwise_weights = relational_matrix[np.ix_(current_states, current_states)] # Compute the pairwise weights by indexing the relational_matrix
# Handle 2-tuples separately, where we directly subtract log of the weights
i_upper, j_upper = np.triu_indices(num_vertices, k=1)  # Get indices for all 2-tuples (upper triangle)
energy -= np.sum(np.log(pairwise_weights[i_upper, j_upper]))  # Subtract log of all pairwise weights

print(i_upper, j_upper)
print(pairwise_weights)


[[1.         0.37262467 0.13818001 0.69311656]
 [0.37262467 1.         0.5142627  0.48602117]
 [0.13818001 0.5142627  1.         0.73740158]
 [0.69311656 0.48602117 0.73740158 1.        ]]
[0 0 1] [1 2 2]
[[1.         0.37262467 0.13818001]
 [0.37262467 1.         0.5142627 ]
 [0.13818001 0.5142627  1.        ]]


In [14]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
relational_probabilities = generate_symmetric_matrix(4)
print("\nRandom Relational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Generate random probability distributions for 3 bounding boxes and 4 states (this should come from the Computer Vision observation)
bounding_box_probabilities = generate_random_probabilities(num_boxes=3, num_states=4)
print("\nRandom Bounding Box Probabilities (Computer Vision):")
print(bounding_box_probabilities, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, bounding_box_probabilities)  # Use the bounding box probabilities as the observation

# Run inference using the Uf function
result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")

print("\nFinal RMRF after inference (Bounding Box States):")
print(result)  # This will be a 3x1 matrix with the final state for each bounding box

Initial RMRF
 [[0]
 [0]
 [0]]

Random Relational Probability Matrix (ConceptNet):
[[1.         0.37262467 0.13818001 0.69311656]
 [0.37262467 1.         0.5142627  0.48602117]
 [0.13818001 0.5142627  1.         0.73740158]
 [0.69311656 0.48602117 0.73740158 1.        ]]

Random Bounding Box Probabilities (Computer Vision):
[[0.29079279 0.07427418 0.20491863 0.4300144 ]
 [0.42070478 0.01251235 0.30446055 0.26232231]
 [0.40747266 0.15727423 0.28766843 0.14758468]] 

Succesfully finish, iteration: 0

Final RMRF after inference (Bounding Box States):
[[0]
 [0]
 [0]]


In [14]:
# Define the states
s = [0, 1, 2, 3]  # States corresponding to ["tree", "glass", "car", "cup"]

# Initialize the RMRF as a vector of length 3 (one component for each bounding box)
r = np.zeros((3, 1), dtype=int)  # Initial RMRF values (three rows, one column)
print("Initial RMRF\n", r)

# Assuming we have a 4x4 relational probability matrix for the 4 states (this should come from the ConceptNet)
# Relational probability matrix (ConceptNet-like)
relational_probabilities = np.array([
    [1.0, 0.9, 0.95, 0.2],  # State 0 ("tree") is highly related to State 1 ("glass") and State 2 ("car")
    [0.9, 1.0, 0.95, 0.3],  # State 1 ("glass") is highly related to State 0 ("tree") and State 2 ("car")
    [0.95, 0.95, 1.0, 0.4], # State 2 ("car") is highly related to State 0 ("tree") and State 1 ("glass")
    [0.2, 0.3, 0.4, 1.0]    # State 3 ("cup") is weakly related to other states
])

print("\nRelational Probability Matrix (ConceptNet):")
print(relational_probabilities)

# Bounding Box Probabilities (Computer Vision)
# Strong preference for state 0 for bounding box 0, state 1 for bounding box 1, and state 2 for bounding box 2
observation = np.array([
    [0.9, 0.05, 0.03, 0.02],  # Bounding Box 0 (strongly prefers "tree")
    [0.1, 0.8, 0.05, 0.05],   # Bounding Box 1 (strongly prefers "glass")
    [0.05, 0.1, 0.75, 0.1]    # Bounding Box 2 (strongly prefers "car")
])
print("\nBounding Box Probabilities (Computer Vision):")
print(observation, "\n")

# Create an instance of RMRFwO and run inference
mr = mrf(s, r, observation)  # Use the bounding box probabilities as the observation

Initial RMRF
 [[0]
 [0]
 [0]]

Relational Probability Matrix (ConceptNet):
[[1.   0.9  0.95 0.2 ]
 [0.9  1.   0.95 0.3 ]
 [0.95 0.95 1.   0.4 ]
 [0.2  0.3  0.4  1.  ]]

Bounding Box Probabilities (Computer Vision):
[[0.9  0.05 0.03 0.02]
 [0.1  0.8  0.05 0.05]
 [0.05 0.1  0.75 0.1 ]] 



In [15]:
# Calculate the energy for all possible assignments
def calculate_all_assignments_energy():
    # Generate all possible state assignments for 3 bounding boxes and 4 states
    all_assignments = list(it.product([0, 1, 2, 3], repeat=3))

    # Initialize results to store the energies for all assignments
    results = []

    # Iterate through all possible assignments
    for assignment in all_assignments:
        # Convert assignment to a 2D array to be compatible with Uf function
        rmrf = np.array(assignment).reshape(-1, 1)

        # Calculate the total energy for the current assignment
        total_energy = 0.0
        for row in range(rmrf.shape[0]):
            total_energy += Uf(rmrf, observation, row, col=0)  # HALF THE WEIGHTS BECAUSE THEY ARE ADDED TWICE
        
        # Store the assignment and its total energy
        results.append((assignment, total_energy))
    
    # Convert results to a DataFrame for easy visualization
    df = pd.DataFrame(results, columns=["Assignment", "Total Energy"])

    # Add a new column 'Rank' that gives the order of the counts
    df["Rank"] = df["Total Energy"].rank(method="min").astype(int)
    
    return df

# Calculate energies for all assignments
all_assignment_energies = calculate_all_assignments_energy()

# Display the result in a sorted manner (ascending by energy)
all_assignment_energies_sorted = all_assignment_energies.sort_values(by="Total Energy")
print(all_assignment_energies_sorted.head(10))  # Display the top results to check the minimum energy

   Assignment  Total Energy  Rank
6   (0, 1, 2)      1.239059     1
2   (0, 0, 2)      3.002506     2
5   (0, 1, 1)      3.259510     3
10  (0, 2, 2)      3.695653     4
22  (1, 1, 2)      3.813436     5
4   (0, 1, 0)      3.952657     6
38  (2, 1, 2)      4.324261     7
1   (0, 0, 1)      5.338951     8
0   (0, 0, 0)      5.403678     9
21  (1, 1, 1)      5.521461    10


In [16]:
# Run the inference n times and display the results for each run and the counts for each unique result
n = 10000  # Number of inference runs
results = []

# Create a dummy stream to suppress print statements
f = io.StringIO()

for i in range(n):
    # Create an instance of RMRFwO and run inference
    mr = mrf(s, r, observation)  # Use the bounding box probabilities as the observation
    
    # Suppress the output of inference method
    with contextlib.redirect_stdout(f):
        result = mr.inference(Uf=Uf, maxIterations=100, Temp=1.0, tempReduction=1.0, optimal="MAP")
    
    # Store the result
    results.append(list(result.flatten()))  # Convert to a list of integers for proper comparison

# Count the occurrences of each unique result
unique_results, counts = np.unique(results, axis=0, return_counts=True)

# Create a DataFrame to display the results
df_results = pd.DataFrame({
    'Result': [str(res) for res in unique_results],  # Convert the result tuples to strings for better display
    'Count': counts,
    'Percentage': (counts / n) * 100
})

# Add a new column 'Rank' that gives the order of the counts
df_results['Rank'] = df_results['Count'].rank(ascending=False, method='dense').astype(int)

# Sort the results by count
df_results = df_results.sort_values(by='Count', ascending=False)

# Display the DataFrame
print(df_results.head(10))

    Result  Count  Percentage  Rank
5  [0 1 2]   3263       32.63     1
0  [0 0 0]   2245       22.45     2
2  [0 0 2]   1791       17.91     3
4  [0 1 1]   1377       13.77     4
1  [0 0 1]    961        9.61     5
3  [0 1 0]    363        3.63     6


In [7]:
# Define the search endpoint for fetching edges with the 'AtLocation' relation
search_url = 'http://api.conceptnet.io/query'

# Function to search for edges with 'AtLocation' relation, including 'glass' as both start and end
def get_atlocation_edges_both_directions(concept, max_limit=50):
    params_start = {
        'start': f'/c/en/{concept}',  # Fetch edges where 'glass' is the start
        'rel': '/r/AtLocation',
        'limit': max_limit
    }
    params_end = {
        'end': f'/c/en/{concept}',    # Fetch edges where 'glass' is the end
        'rel': '/r/AtLocation',
        'limit': max_limit
    }

    # Fetch edges where 'glass' is the start
    response_start = requests.get(search_url, params=params_start).json()
    edges_start = response_start['edges']

    # Fetch edges where 'glass' is the end
    response_end = requests.get(search_url, params=params_end).json()
    edges_end = response_end['edges']

    # Combine both sets of edges
    return edges_start + edges_end

# Fetch 'AtLocation' edges for the concept "glass" with a limit
atlocation_edges = get_atlocation_edges_both_directions('glass', max_limit=100000)

# Print the number of 'AtLocation' edges retrieved
print(f"Total 'AtLocation' edges retrieved for 'glass': {len(atlocation_edges)}")

#Show all the weight and concepts for AtLocation of glas and show diretcion in a visual way
for edge in atlocation_edges:
    start = edge['start']['label'].split('/')[-1]
    end = edge['end']['label'].split('/')[-1]
    weight = edge['weight']
    print(f"{start} -> {end} (Weight: {weight})")

Total 'AtLocation' edges retrieved for 'glass': 17
a glass -> the cabinet (Weight: 4.0)
a glass -> a window (Weight: 3.4641016151377544)
a glass -> a kitchen (Weight: 2.82842712474619)
a glass -> the table (Weight: 2.0)
a glass -> a water cooler (Weight: 1.0)
a glass -> a dishwasher (Weight: 1.0)
a glass -> a hand (Weight: 1.0)
a glass -> a sink (Weight: 1.0)
a glass -> a cuboard (Weight: 1.0)
a glass -> your kitchen cupboard (Weight: 1.0)
a glass -> a (Weight: 1.0)
a glass -> a dining room (Weight: 1.0)
glass -> a street (Weight: 1.0)
water -> a glass (Weight: 4.0)
an overflow -> a glass (Weight: 1.0)
ice -> a glass (Weight: 1.0)
ice cubes -> a glass (Weight: 1.0)


In [8]:
# Define the list of concepts in the desired order
concepts = ["tree", "glass", "car", "cup"]

# Define the base URL for the ConceptNet relatedness API
relatedness_url = 'http://api.conceptnet.io/relatedness'

# Function to get the relatedness value between two concepts
def get_relatedness(concept1, concept2):
    params = {
        'node1': f'/c/en/{concept1}',
        'node2': f'/c/en/{concept2}'
    }
    response = requests.get(relatedness_url, params=params).json()
    return response['value']

# Initialize a 4x4 matrix to store the relatedness values
relatedness_matrix = np.ones((4, 4))  # Start with an identity matrix (relatedness of 1 for the same concepts)

# Create all concept pairs (excluding same concept pairs as they are 1.0 by default)
concept_pairs = list(it.product(range(4), repeat=2))
concept_pairs = [(i, j) for i, j in concept_pairs if i != j]

# Fetch relatedness for each pair and populate the matrix
for i, j in concept_pairs:
    relatedness_matrix[i, j] = get_relatedness(concepts[i], concepts[j])

# Rescale the relatedness values from [-1, 1] to [0, 1]
rescaled_matrix = (relatedness_matrix + 1) / 2

# Create a pandas DataFrame for better readability with the desired order
relatedness_df = pd.DataFrame(rescaled_matrix, index=concepts, columns=concepts)

# Display the rescaled 4x4 matrix
print(relatedness_df)

         tree   glass     car     cup
tree   1.0000  0.5315  0.5450  0.4920
glass  0.5315  1.0000  0.5365  0.6345
car    0.5450  0.5365  1.0000  0.5180
cup    0.4920  0.6345  0.5180  1.0000
